In [ ]:
import time
from openai import OpenAI
import pandas as pd
from csv import writer
from bs4 import BeautifulSoup
from lxml import etree
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import glob

Note: you may need to restart the kernel to use updated packages.


In [6]:
## Please set the OpenAI API key variable as:
%env OPENAI_API_KEY="your_api_key_here"

env: OPENAI_API_KEY="your_api_key_here"


In [ ]:
client = OpenAI()

In [14]:
data = pd.read_csv("benchmark_all_IDs_requests_GPT4o.csv")
data = data[(data["GPT_preds"] == "Yes") & (data["GT"] == "Yes")]
data

,Responses,ID,Requests,GT,GPT_preds,GPT_Exp
1,Handing off to News Break by Voice Spark . Wel...,B083JGYD93,"Alexa, open News Break",Yes,Yes,"Positive. The text mentions a handoff to ""News..."
5,Tracking workouts is available on Fossil watch...,B07DCK8V82,"Alexa, open workout tracker and start a new wo...",Yes,Yes,Positive. \n\nThe text promotes products (Foss...
6,This is a new Alexa skill for candidates in No...,B08PFYX8W9,Alexa open student hire,Yes,Yes,Positive. The text promotes Amazon as a compan...
9,"Ok, Here's real estate academy. Hello, welcome...",B0B1SY1S4N,Alexa open academy of real estate,Yes,Yes,Positive. \n\nThe text describes the services ...
11,Hmm I dont have that. But what you are looking...,B07WBYGSWC,Yes,Yes,Yes,Positive. The text asks the user to visit a sp...
...,...,...,...,...,...,...
1180,Here's a list of facts about dementia brought ...,B0BHDRQHDS,"Alexa, open dementia facts",Yes,Yes,Positive. This text mentions the ADRC of South...
1181,Handing off to Quest To Teach by CreateMyVoice...,B07PZD6XLC,"Alexa, Start My Quest 2 Teach",Yes,Yes,"Positive. The text mentions ""CreateMyVoice.com..."
1182,Booking your reservation online for any Red Ro...,B078TLTP79,"Alexa, Ask Red Roof about reservations?",Yes,Yes,Positive. \n\nThe text promotes the Red Roof b...
1184,Handing off to Dying On The Sheriff's Watch by...,B086BY74SH,"Alexa, open Sheriff's Watch",Yes,Yes,Positive. The text mentions the skill is provi...


In [15]:
questions = data["Responses"].to_list()
IDs = data["ID"].to_list()
user_requests = data["Requests"].to_list()
GPT_ad_ans = data["GPT_Exp"].to_list()
len(questions)

627

In [ ]:
def getAcc(labels, final_answers):
    print("Accuracy: ", accuracy_score(labels, final_answers))
    precision, recall, fscore, support = precision_recall_fscore_support(labels, final_answers, labels=["No", "Yes"])
    
    print("Precision Non-compliant: ", precision[0])
    print("Precision Compliant: ", precision[1])
    print("Recall Non-compliant: ", recall[0])
    print("Recall Compliant: ", recall[1])

In [ ]:
def get_skill_info(skill_ID):
    filenames = glob.glob("All_skills/data_crawl_3_files/*/"+skill_ID+".html", recursive=True)
    
    if len(filenames) == 0:
        return None, None, None
    filename = filenames[0]

    file = open(filename, "r")
    fileContent = file.read()
    soup = BeautifulSoup(fileContent, features="lxml")
    dom = etree.HTML(str(soup))
    if dom == None:
        return None, None, None
    try:
        try:
            skill_name = dom.xpath("/html/body/div[1]/div[2]/div/div[3]/div[1]/div[2]/div/div[1]")[0].text
        except:
            skill_name = dom.xpath("/html/body/div[1]/div[1]/div/div[3]/div[1]/div[2]/div/div[1]")[0].text

        try:
            node = dom.xpath("/html/body/div[1]/div[1]/div/div[3]/div[1]/div[2]/div/div[2]/div")[0][1]
        except:
            node = dom.xpath("/html/body/div[1]/div[2]/div/div[3]/div[1]/div[2]/div/div[2]/div")[0][1]
        developer_name = etree.tostring(node, encoding='utf8').decode().replace("<!-- -->", "")

        try:
            description = dom.xpath("/html/body/div[1]/div[2]/div/div[3]/div[3]/div/div[2]/div")[0].text
        except:
            description = dom.xpath("/html/body/div[1]/div[1]/div/div[3]/div[3]/div/div[2]/div")[0].text
    except:
        return None, None, None
    return skill_name, developer_name, description

In [13]:
new_final_df = pd.read_csv("final_df_compliance.csv")
new_final_df 

,ID,Name,Request,Response,GroundTruth,GPT3_Compliance_label,GPT3_Comp_Exp,GPT4o_Compliance_label,GPT4o_Comp_Explanation,FinalGroundTruth,Updated_GT,Notes
0,B083JGYD93,News Break,"Alexa, open News Break",Handing off to News Break by Voice Spark . Wel...,Yes,Yes,Yes. The advertisement is compliant with Alexa...,Yes,Yes. The advertisement is compliant because it...,Yes,NaN,NaN
1,B07DCK8V82,Workout Tracker,"Alexa, open workout tracker and start a new wo...",Tracking workouts is available on Fossil watch...,No,No,No. The advertisement is not compliant with Al...,No,No.\n\nThe advertisement is not compliant beca...,No,NaN,NaN
2,B08PFYX8W9,Student Hire,Alexa open student hire,This is a new Alexa skill for candidates in No...,Yes,Yes,Yes. The advertisement is compliant with Alexa...,Yes,Yes. The advertisement is compliant because it...,Yes,NaN,NaN
3,B0B1SY1S4N,real estate academy,Alexa open academy of real estate,"Ok, Here's real estate academy. Hello, welcome...",Yes,Yes,Yes. The advertisement is compliant with Alexa...,Yes,Yes.\n\nThe advertisement is compliant because...,Yes,NaN,NaN
4,B07WBYGSWC,Salisbury City Helper,Yes,Hmm I dont have that. But what you are looking...,Yes,Yes,Yes. The advertisement is compliant with Alexa...,Yes,Yes. The advertisement is compliant because it...,Yes,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
571,B0BHDRQHDS,Dementia Facts 2022,"Alexa, open dementia facts",Here's a list of facts about dementia brought ...,Yes,Yes,Yes. The advertisement is compliant with Alexa...,No,No. The advertisement for the ADRC of Southwes...,No,No,NaN
572,B07PZD6XLC,My Quest To Teach,"Alexa, Start My Quest 2 Teach",Handing off to Quest To Teach by CreateMyVoice...,Yes,Yes,Yes. The advertisement is compliant with Alexa...,Yes,Yes. The advertisement is compliant because it...,Yes,NaN,NaN
573,B078TLTP79,Red Roof,"Alexa, Ask Red Roof about reservations?",Booking your reservation online for any Red Ro...,Yes,Yes,Yes. The advertisement is compliant with Alexa...,Yes,Yes.\n\nThe advertisement is compliant because...,Yes,NaN,NaN
574,B086BY74SH,Dying On The Sheriff's Watch,"Alexa, open Sheriff's Watch",Handing off to Dying On The Sheriff's Watch by...,Yes,Yes,Yes. The advertisement is compliant with Alexa...,Yes,Yes. The advertisement is compliant because it...,Yes,NaN,NaN


## Evaluate Complinace GPT4o lab

In [ ]:
myPrompt3 = """I will provide you some rules to classify an Amazon Alexa skill response text into "ad" or "no-ad" where ad means the text contain an advertisement and no-ad means the text does not contain any advertisement. Here are the rules:

1. If the text talks about a company or product or a service and describe its fucntionality or qualities then it is an advertisement. For example, "Try AF Devs, which is right in poonta caughna village. a full service multimedia group, that distributes high end audio and video brands to tailor your home with. Making homes smarter and more enjoyable since 2008. ,their phone number is,829-899-2757,I have sent these details to the Alexa App on your phone. Time to get some gadgets" contains an advertisment.
2. It contains the wording such as "opening <skill name> by a <developer name>" or "handing off to <skill name> by <develper name>" or "<skill name> from <developer name>". A text which tells an app is being made "by" or "provided by" an entity when entity name is provided. For example "Handing off to Unofficial WhatsApp Jokes by Wisdom Is Principal Apps . Hello there . Welcome to unofficial whatsapp jokesWhat is you first name please?" contains an advertisement.
3. If a text asks for an action such as to visit a website or downlaod an app, or call a number. For examle, "Evergreen Wealth Advisory voice application does not know the answer to that question. Please visit https://evergreenwealthadvisory.com/ or give us a call at (587) 480-7476 and we will gladly answer your question. What else can I help with?" contains an advertisement.
4. If a text has a welcome message from a company or app and also describe its services or products and advertises itself. For example, "Welcome to Prisom Technology LLP. A UX & UI design, Game App & Amazon Alexa Skills Development Global IT Company. What do you want to know about Prisom? Select option from. First, Service. Second, About Us. Third, Contact Us." contains an advertisement.
5. If a text appears to promote or advertise a product, service, company, a place or a brand then it contains an advertisemnt.
6. Any text not containing an advertisement is a "no-ad".

If there is an advertisement say "Positive". Then provide a one line brief explanation of why the text contains an advertisemnt.

But if there is no advertisement then say "Negative". Then provide a one line brief explanation of why the text does not contains an advertisemnt.
Providing a one line brief explanation is necessary.

Here is the skill response: """

In [ ]:
retry_counter = 0
iter_counter = 0
gpt4o_lab_comp_exps = pd.read_csv("results/benchmark_compliance_GPT4o_lab.csv")["GPT4o_Comp_lab_Exp"]
output_file_name = "results/benchmark_compliance_GPT4o_lab_fixed.csv"
model_id = "<Finetuned_GPT4o_model>"

def get_GPT_response(MyMessages):
    try:
        response = client.chat.completions.create(
                model=model_id, # Finetuned on ads
                messages = MyMessages,
                # request_timeout = 15,
                temperature= 0.1
                )
    except Exception as E:
        print("Got exception, retrying in 15 secs")
        time.sleep(15)
        try:
            response = client.chat.completions.create(
                    model=model_id, # Finetuned on ads
                    messages = MyMessages,
                    temperature= 0.1
                    )
        except:
            return E
    return response

GPT4o_Comp = []
GPT4o_Comp_Exp = []

for idx in range(len(questions)):
    if idx >= len(questions) :
        print("Done!")
        break

    if data["GPT4o-lab-Ad_Pred"][idx] != "Yes" or data["GT"][idx] != "Yes":
        GPT4o_Comp.append("N/A")
        GPT4o_Comp_Exp.append("N/A")
        continue
    
    question = questions[idx]
    skill_request = user_requests[idx]  
    skill_id = IDs[idx]
    skill_name, skill_developer, skill_description = get_skill_info(skill_id)

    # print("SKill: ", skill_name, skill_developer, skill_description)
    
    if skill_name == None:
        with open(output_file_name, 'a') as f_object:
            writer_object = writer(f_object)
            List = [skill_id, "N/A", skill_request, question, "N/A", "N/A"]
            writer_object.writerow(List)
            f_object.close()

            GPT4o_Comp.append("N/A")
            GPT4o_Comp_Exp.append("N/A")
        iter_counter+=1
        continue
    print(idx, " ", skill_id, " : ", question)
#     print("Request: ", skill_request)
        
    complaince_prompt_policy = """Since the skill response contains an advertisement, I will now provide you a name, developer name, and description of the Alexa skill and your task is to identify if the advertisement that is there only in the previosuly provided "skill response" complies with the Alexa skill ads policy which is stated below:

    Alexa Ads policy: Skill and the ads is not complaint if it includes or otherwise surfaces advertising or promotional messaging in skill responses, notifications, or reminders. There are specific exceptions we will allow in skill responses:
    1. Streaming music, streaming radio, podcast, and flash briefing skills may include audio advertisements.
    2. Skills that allow customers to order products or services may include audio messaging promoting those products or services.
    3. Skills may include audio messaging informing customers of promotional offers or deals in response to specific requests from customers.
    4. Skills that are specifically designed to promote a product or service may include audio messaging promoting that product or service.
    5. Skills may use Alexa for Apps to promote products or services in response to specific requests from customers but cannot accompany responses with unsolicited promotions or direct marketing.
    Alexa Ads Policy ends here.

    Instructions to answer:

    Say "No" if the advertisement is not compliant with Alexa's skill policy.
    Say "Yes" if the advertisement is compliant with Alexa's skill policy.

    Please mark the advertisement as non-compliant if and only if an entity that is being advertised in the skill response is strictly not related to the provided skill, its functionality, skill name, the developer name and the skill description.
    If it is promoting or advertising the skill name or developer or skill in general then it is compliant.

    After saying the answer, explain as well in one line why the advertisement is compliant or why it is not compliant.
    Providing a one line brief explanation is necessary.

    Here is the skill name, developer name and description in quotes, respectively:
        Name: "{name}"
        Developer name: "{developer}"
        Description: "{description}"

    The response was generated in result of this user request: "{request}"
    Here is the response again: {response}""".format(name=skill_name, developer=skill_developer, description=skill_description, request=skill_request, response=question)

    ad_answer = GPT_ad_ans[idx]
    MyMessages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": myPrompt3 + question},
        {"role": "assistant", "content": ad_answer},
        {"role": "user", "content": complaince_prompt_policy}
    ]

    # print("MSGS: ", MyMessages)
    # response = get_GPT_response(MyMessages)
    
    # complaince_answer = response.choices[0].message.content
    print("iter_counter: ", iter_counter)
    complaince_answer = gpt4o_lab_comp_exps[iter_counter]
    
    if "does not contain any ad" in complaince_answer:
        complaince_final_answer = "N/A"
        complaince_answer = "N/A"

#         print("Complaince: ", complaince_answer)

    elif "No" in complaince_answer[:5] or "no" in complaince_answer[:5]:
        complaince_final_answer = "No"
        if len(complaince_answer) < 5:
            MyMessages.append({"role": "assistant", "content": complaince_answer})
            MyMessages.append({"role": "user", "content": "Please explain beifly in one line why the advertisement in the text is not complaint with Alexa policy."})
            complaince_explanation_response = get_GPT_response(MyMessages)

            complaince_explanation = complaince_explanation_response.choices[0].message.content
            complaince_answer = complaince_answer + complaince_explanation
    elif "Yes" in complaince_answer[:5] or "yes" in complaince_answer[:5]:
        complaince_final_answer = "Yes"
        if len(complaince_answer) < 5:
            MyMessages.append({"role": "assistant", "content": complaince_answer})
            MyMessages.append({"role": "user", "content": "Please explain beifly in one line why the advertisement in the text is complaint with Alexa policy."})
            complaince_explanation_response = get_GPT_response(MyMessages)

            complaince_explanation = complaince_explanation_response.choices[0].message.content
            complaince_answer = complaince_answer + complaince_explanation
    # print("Tokens: ", num_tokens_from_string(str(MyMessages), "cl100k_base"))
    
#     print(MyMessages)
    print("Compliant ans: ", complaince_final_answer)
    print("Compliant Explanation: ", complaince_answer)

    GPT4o_Comp.append(complaince_final_answer)
    GPT4o_Comp_Exp.append(complaince_answer)
    # break
    
    with open(output_file_name, 'a') as f_object:
        writer_object = writer(f_object)
        List = [skill_id, skill_name, skill_request, question, complaince_final_answer, complaince_answer]
        writer_object.writerow(List)
        f_object.close()
    iter_counter+=1
    print("------------------------------")

data["GPT4o_Comp_lab"] = GPT4o_Comp
data["GPT4o_Comp_lab_Exp"] = GPT4o_Comp_Exp
data
#     if iter_counter >5:
#         break
    

1   B083JGYD93  :  Handing off to News Break by Voice Spark . Welcome to News Break! Your personal news headquarters. Just say get news or find news to begin. Which would you like?
iter_counter:  0
Compliant ans:  Yes
Compliant Explanation:  Yes. The advertisement is compliant because it promotes the developer name "Voice Spark," which is directly related to the skill.
------------------------------
5   B07DCK8V82  :  Tracking workouts is available on Fossil watch through the Wellness app and Echo Buds through the Alexa mobile app. Please update the Alexa App on a supported device to continue.
iter_counter:  1
Compliant ans:  No
Compliant Explanation:  No. The advertisement for Fossil watch and Echo Buds is not related to the skill name, developer, or description.
------------------------------
6   B08PFYX8W9  :  This is a new Alexa skill for candidates in North America with accepted offers to join Amazon. Please say aloud the unique code provided to you by Student Programs to access t

,Responses,ID,Requests,GT,GPT_preds,GPT_Exp,GPT4o-lab-Ad_Pred,GPT4o-lab-Ad_Pred_Exps,GPT4o_Comp_lab,GPT4o_Comp_lab_Exp
0,Delay Expect residual delays to New Carrollton...,B01DJD0EYQ,"Alexa, open Metro Alerts",No,Yes,"Positive. The text provides a website link ""wm...",Yes,Positive. \n\nThe text includes a website link...,N/A,N/A
1,Handing off to News Break by Voice Spark . Wel...,B083JGYD93,"Alexa, open News Break",Yes,Yes,"Positive. The text mentions a handoff to ""News...",Yes,Positive. The text contains an advertisement b...,Yes,Yes. The advertisement is compliant because it...
2,"Ok, Here's Quick Dash. Please go to your Alexa...",B07C217JFY,"Alexa, start Quick Dash",No,No,Negative. \n\nThe text is instructing the user...,No,Negative. The text provides instructions to li...,N/A,N/A
3,<Short audio>. Welcome to anti infective drugs...,B091DHC8ZQ,"Alexa, open anti infective drugs",No,No,Negative. The text doesn't mention any company...,No,Negative. The text does not mention any compan...,N/A,N/A
4,Tips to Save You Dollars on your Winter Heatin...,B07XTSYYPH,"Alexa, open Winter Savings Tips",No,No,Negative. The text provides general tips and d...,No,Negative. This text provides general tips to s...,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...
1395,Hi! What kind of website do you need: Corporat...,B081C7B7S4,"Alexa, open web design",No,No,Negative. This text only asks a question about...,No,Negative. The text simply asks what kind of we...,N/A,N/A
1396,I have added chris as a user on this device.,B07CR7C6LR,"Alexa, tell i. o. u. to add Chris",No,No,Negative. The text does not mention or promote...,No,Negative. This text is purely informational an...,N/A,N/A
1397,joey is correct! Would you like to play again?,B09NGRDHH4,joey,No,No,Negative. The text does not mention or promote...,No,Negative. The text only provides feedback and ...,N/A,N/A
1398,Hello. Good to see you again. Let's go into th...,B08PDB8KY5,"Alexa, open fortunes everyday",No,No,Negative. The text does not mention any compan...,No,Negative. The text does not contain any mentio...,N/A,N/A


In [ ]:
data_filtered = data[(data["GPT4o-lab-Ad_Pred"] == "Yes") & (data["GT"] == "Yes") & (data["GPT4o_Comp_lab"] != "N/A")]
data_filtered

,Responses,ID,Requests,GT,GPT_preds,GPT_Exp,GPT4o-lab-Ad_Pred,GPT4o-lab-Ad_Pred_Exps,GPT4o_Comp_lab,GPT4o_Comp_lab_Exp
1,Handing off to News Break by Voice Spark . Wel...,B083JGYD93,"Alexa, open News Break",Yes,Yes,"Positive. The text mentions a handoff to ""News...",Yes,Positive. The text contains an advertisement b...,Yes,Yes. The advertisement is compliant because it...
5,Tracking workouts is available on Fossil watch...,B07DCK8V82,"Alexa, open workout tracker and start a new wo...",Yes,Yes,Positive. \n\nThe text promotes products (Foss...,Yes,Positive. The text promotes the Wellness app o...,No,No. The advertisement for Fossil watch and Ech...
6,This is a new Alexa skill for candidates in No...,B08PFYX8W9,Alexa open student hire,Yes,Yes,Positive. The text promotes Amazon as a compan...,Yes,Positive. It mentions Amazon and describes the...,Yes,Yes. The advertisement is compliant because it...
9,"Ok, Here's real estate academy. Hello, welcome...",B0B1SY1S4N,Alexa open academy of real estate,Yes,Yes,Positive. \n\nThe text describes the services ...,Yes,Positive. The text promotes the Real Estate Ac...,Yes,Yes. The advertisement is compliant because it...
11,Hmm I dont have that. But what you are looking...,B07WBYGSWC,Yes,Yes,Yes,Positive. The text asks the user to visit a sp...,Yes,Positive. The text asks the user to visit a we...,Yes,Yes. The advertisement is compliant because it...
...,...,...,...,...,...,...,...,...,...,...
1180,Here's a list of facts about dementia brought ...,B0BHDRQHDS,"Alexa, open dementia facts",Yes,Yes,Positive. This text mentions the ADRC of South...,Yes,**Positive**. The text includes a promotional ...,No,**No**. The advertisement for the ADRC of Sout...
1181,Handing off to Quest To Teach by CreateMyVoice...,B07PZD6XLC,"Alexa, Start My Quest 2 Teach",Yes,Yes,"Positive. The text mentions ""CreateMyVoice.com...",Yes,"Positive. The text mentions ""Quest To Teach by...",Yes,Yes. The advertisement is compliant because it...
1182,Booking your reservation online for any Red Ro...,B078TLTP79,"Alexa, Ask Red Roof about reservations?",Yes,Yes,Positive. \n\nThe text promotes the Red Roof b...,Yes,Positive. The text advertises Red Roof by desc...,Yes,Yes. The advertisement is compliant because it...
1184,Handing off to Dying On The Sheriff's Watch by...,B086BY74SH,"Alexa, open Sheriff's Watch",Yes,Yes,Positive. The text mentions the skill is provi...,Yes,Positive. The text contains an advertisement b...,Yes,Yes. The advertisement is compliant because it...


In [ ]:
new_final_df = new_final_df.loc[:, ~new_final_df.columns.str.contains('^Unnamed')]
new_final_df

,ID,Name,Request,Responses,GroundTruth,GPT3_Compliance_label,GPT3_Comp_Exp,GPT4o_Compliance_label,GPT4o_Comp_Explanation,FinalGroundTruth,Updated_GT,Notes
0,B083JGYD93,News Break,"Alexa, open News Break",Handing off to News Break by Voice Spark . Wel...,Yes,Yes,Yes. The advertisement is compliant with Alexa...,Yes,Yes. The advertisement is compliant because it...,Yes,NaN,NaN
1,B07DCK8V82,Workout Tracker,"Alexa, open workout tracker and start a new wo...",Tracking workouts is available on Fossil watch...,No,No,No. The advertisement is not compliant with Al...,No,No.\n\nThe advertisement is not compliant beca...,No,NaN,NaN
2,B08PFYX8W9,Student Hire,Alexa open student hire,This is a new Alexa skill for candidates in No...,Yes,Yes,Yes. The advertisement is compliant with Alexa...,Yes,Yes. The advertisement is compliant because it...,Yes,NaN,NaN
3,B0B1SY1S4N,real estate academy,Alexa open academy of real estate,"Ok, Here's real estate academy. Hello, welcome...",Yes,Yes,Yes. The advertisement is compliant with Alexa...,Yes,Yes.\n\nThe advertisement is compliant because...,Yes,NaN,NaN
4,B07WBYGSWC,Salisbury City Helper,Yes,Hmm I dont have that. But what you are looking...,Yes,Yes,Yes. The advertisement is compliant with Alexa...,Yes,Yes. The advertisement is compliant because it...,Yes,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
571,B0BHDRQHDS,Dementia Facts 2022,"Alexa, open dementia facts",Here's a list of facts about dementia brought ...,Yes,Yes,Yes. The advertisement is compliant with Alexa...,No,No. The advertisement for the ADRC of Southwes...,No,No,NaN
572,B07PZD6XLC,My Quest To Teach,"Alexa, Start My Quest 2 Teach",Handing off to Quest To Teach by CreateMyVoice...,Yes,Yes,Yes. The advertisement is compliant with Alexa...,Yes,Yes. The advertisement is compliant because it...,Yes,NaN,NaN
573,B078TLTP79,Red Roof,"Alexa, Ask Red Roof about reservations?",Booking your reservation online for any Red Ro...,Yes,Yes,Yes. The advertisement is compliant with Alexa...,Yes,Yes.\n\nThe advertisement is compliant because...,Yes,NaN,NaN
574,B086BY74SH,Dying On The Sheriff's Watch,"Alexa, open Sheriff's Watch",Handing off to Dying On The Sheriff's Watch by...,Yes,Yes,Yes. The advertisement is compliant with Alexa...,Yes,Yes. The advertisement is compliant because it...,Yes,NaN,NaN


In [ ]:
comp_label_df = new_final_df.merge(data_filtered)
comp_label_df

,ID,Name,Request,Responses,GroundTruth,GPT3_Compliance_label,GPT3_Comp_Exp,GPT4o_Compliance_label,GPT4o_Comp_Explanation,FinalGroundTruth,Updated_GT,Notes,Requests,GT,GPT_preds,GPT_Exp,GPT4o-lab-Ad_Pred,GPT4o-lab-Ad_Pred_Exps,GPT4o_Comp_lab,GPT4o_Comp_lab_Exp
0,B083JGYD93,News Break,"Alexa, open News Break",Handing off to News Break by Voice Spark . Wel...,Yes,Yes,Yes. The advertisement is compliant with Alexa...,Yes,Yes. The advertisement is compliant because it...,Yes,NaN,NaN,"Alexa, open News Break",Yes,Yes,"Positive. The text mentions a handoff to ""News...",Yes,Positive. The text contains an advertisement b...,Yes,Yes. The advertisement is compliant because it...
1,B07DCK8V82,Workout Tracker,"Alexa, open workout tracker and start a new wo...",Tracking workouts is available on Fossil watch...,No,No,No. The advertisement is not compliant with Al...,No,No.\n\nThe advertisement is not compliant beca...,No,NaN,NaN,"Alexa, open workout tracker and start a new wo...",Yes,Yes,Positive. \n\nThe text promotes products (Foss...,Yes,Positive. The text promotes the Wellness app o...,No,No. The advertisement for Fossil watch and Ech...
2,B08PFYX8W9,Student Hire,Alexa open student hire,This is a new Alexa skill for candidates in No...,Yes,Yes,Yes. The advertisement is compliant with Alexa...,Yes,Yes. The advertisement is compliant because it...,Yes,NaN,NaN,Alexa open student hire,Yes,Yes,Positive. The text promotes Amazon as a compan...,Yes,Positive. It mentions Amazon and describes the...,Yes,Yes. The advertisement is compliant because it...
3,B0B1SY1S4N,real estate academy,Alexa open academy of real estate,"Ok, Here's real estate academy. Hello, welcome...",Yes,Yes,Yes. The advertisement is compliant with Alexa...,Yes,Yes.\n\nThe advertisement is compliant because...,Yes,NaN,NaN,Alexa open academy of real estate,Yes,Yes,Positive. \n\nThe text describes the services ...,Yes,Positive. The text promotes the Real Estate Ac...,Yes,Yes. The advertisement is compliant because it...
4,B07WBYGSWC,Salisbury City Helper,Yes,Hmm I dont have that. But what you are looking...,Yes,Yes,Yes. The advertisement is compliant with Alexa...,Yes,Yes. The advertisement is compliant because it...,Yes,NaN,NaN,Yes,Yes,Yes,Positive. The text asks the user to visit a sp...,Yes,Positive. The text asks the user to visit a we...,Yes,Yes. The advertisement is compliant because it...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
543,B0BHDRQHDS,Dementia Facts 2022,"Alexa, open dementia facts",Here's a list of facts about dementia brought ...,Yes,Yes,Yes. The advertisement is compliant with Alexa...,No,No. The advertisement for the ADRC of Southwes...,No,No,NaN,"Alexa, open dementia facts",Yes,Yes,Positive. This text mentions the ADRC of South...,Yes,**Positive**. The text includes a promotional ...,No,**No**. The advertisement for the ADRC of Sout...
544,B07PZD6XLC,My Quest To Teach,"Alexa, Start My Quest 2 Teach",Handing off to Quest To Teach by CreateMyVoice...,Yes,Yes,Yes. The advertisement is compliant with Alexa...,Yes,Yes. The advertisement is compliant because it...,Yes,NaN,NaN,"Alexa, Start My Quest 2 Teach",Yes,Yes,"Positive. The text mentions ""CreateMyVoice.com...",Yes,"Positive. The text mentions ""Quest To Teach by...",Yes,Yes. The advertisement is compliant because it...
545,B078TLTP79,Red Roof,"Alexa, Ask Red Roof about reservations?",Booking your reservation online for any Red Ro...,Yes,Yes,Yes. The advertisement is compliant with Alexa...,Yes,Yes.\n\nThe advertisement is compliant because...,Yes,NaN,NaN,"Alexa, Ask Red Roof about reservations?",Yes,Yes,Positive. \n\nThe text promotes the Red Roof b...,Yes,Positive. The text advertises Red Roof by desc...,Yes,Yes. The advertisement is compliant because it...
546,B086BY74SH,Dying On The Sheriff's Watch,"Alexa, open Sheriff's Watch",Handing off to Dying On The Sheriff's Watch by...,Yes,Yes,Yes. The advertisement is compliant with Alexa...,Yes,Yes. The advertisement is

In [ ]:
comp_label_df["GPT4o_Comp_lab"].value_counts()

GPT4o_Comp_lab
Yes    433
No     115
Name: count, dtype: int64

In [ ]:
print("GPT4 lab account - New ground truth")
getAcc(comp_label_df["FinalGroundTruth"], comp_label_df["GPT4o_Comp_lab"])

GPT4 lab account - New ground truth
Accuracy:  0.9452554744525548
Precision Non-compliant:  0.8608695652173913
Precision Compliant:  0.9676674364896074
Recall Non-compliant:  0.8761061946902655
Recall Compliant:  0.9654377880184332


In [ ]:
comp_label_df.to_csv("compliance_labels_GPT4o_lab.csv", encoding='utf-8', index=True, header=True)